In [1]:
# prerequisites
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
from torchvision.utils import save_image

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
bs = 100

# MNIST Dataset
# transform = transforms.Compose([
#     transforms.ToTensor(),
#     transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))])
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize([0.5], [0.5])])

train_dataset = datasets.MNIST(root='./mnist_data/', train=True, transform=transform, download=True)
test_dataset = datasets.MNIST(root='./mnist_data/', train=False, transform=transform, download=False)

# Data Loader (Input Pipeline)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=bs, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=bs, shuffle=False)

In [5]:
class Generator(nn.Module):
    def __init__(self, g_input_dim, g_output_dim):
        super(Generator, self).__init__()       
        self.fc1 = nn.Linear(g_input_dim, 256)
        self.fc2 = nn.Linear(self.fc1.out_features, self.fc1.out_features*2)
        self.fc3 = nn.Linear(self.fc2.out_features, self.fc2.out_features*2)
        self.fc4 = nn.Linear(self.fc3.out_features, g_output_dim)
    
    # forward method
    def forward(self, x): 
        x = F.leaky_relu(self.fc1(x), 0.2)
        x = F.leaky_relu(self.fc2(x), 0.2)
        x = F.leaky_relu(self.fc3(x), 0.2)
        return torch.tanh(self.fc4(x))
    
class Discriminator(nn.Module):
    def __init__(self, d_input_dim):
        super(Discriminator, self).__init__()
        self.fc1 = nn.Linear(d_input_dim, 1024)
        self.fc2 = nn.Linear(self.fc1.out_features, self.fc1.out_features//2)
        self.fc3 = nn.Linear(self.fc2.out_features, self.fc2.out_features//2)
        self.fc4 = nn.Linear(self.fc3.out_features, 1)
    
    # forward method
    def forward(self, x):
        x = F.leaky_relu(self.fc1(x), 0.2)
        x = F.dropout(x, 0.3)
        x = F.leaky_relu(self.fc2(x), 0.2)
        x = F.dropout(x, 0.3)
        x = F.leaky_relu(self.fc3(x), 0.2)
        x = F.dropout(x, 0.3)
        return torch.sigmoid(self.fc4(x))

In [6]:
# build network
z_dim = 100
mnist_dim = train_dataset.train_data.size(1) * train_dataset.train_data.size(2)

G = Generator(g_input_dim = z_dim, g_output_dim = mnist_dim).to(device)
D = Discriminator(mnist_dim).to(device)

In [5]:
G

Generator(
  (fc1): Linear(in_features=100, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=512, bias=True)
  (fc3): Linear(in_features=512, out_features=1024, bias=True)
  (fc4): Linear(in_features=1024, out_features=784, bias=True)
)

In [6]:
D

Discriminator(
  (fc1): Linear(in_features=784, out_features=1024, bias=True)
  (fc2): Linear(in_features=1024, out_features=512, bias=True)
  (fc3): Linear(in_features=512, out_features=256, bias=True)
  (fc4): Linear(in_features=256, out_features=1, bias=True)
)

In [7]:
# loss
criterion = nn.BCELoss() 

# optimizer
lr = 0.0002 
G_optimizer = optim.Adam(G.parameters(), lr = lr)
D_optimizer = optim.Adam(D.parameters(), lr = lr)

In [13]:
def D_train(x):
    #=======================Train the discriminator=======================#
    D.zero_grad()

    # train discriminator on real
    x_real, y_real = x.view(-1, mnist_dim), torch.ones(bs, 1)
    x_real, y_real = Variable(x_real.to(device)), Variable(y_real.to(device))
    print("D", x_real.shape, y_real.shape)

    D_output = D(x_real)
    D_real_loss = criterion(D_output, y_real)
    D_real_score = D_output

    # train discriminator on facke
    z = Variable(torch.randn(bs, z_dim).to(device))
    x_fake, y_fake = G(z), Variable(torch.zeros(bs, 1).to(device))
    print("D", x_fake.shape, y_fake.shape)

    D_output = D(x_fake)
    D_fake_loss = criterion(D_output, y_fake)
    D_fake_score = D_output
    print("D", D_fake_score.shape)

    # gradient backprop & optimize ONLY D's parameters
    D_loss = D_real_loss + D_fake_loss
    D_loss.backward()
    D_optimizer.step()
        
    return  D_loss.data.item()

In [15]:
def G_train(x):
    #=======================Train the generator=======================#
    G.zero_grad()

    z = Variable(torch.randn(bs, z_dim).to(device))
    y = Variable(torch.ones(bs, 1).to(device))
    print("G", z.shape, y.shape)

    G_output = G(z)
    D_output = D(G_output)
    G_loss = criterion(D_output, y)
    print("G_loss:", G_loss)

    # gradient backprop & optimize ONLY G's parameters
    G_loss.backward()
    G_optimizer.step()
        
    return G_loss.data.item()

In [16]:
n_epoch = 2
for epoch in range(1, n_epoch+1):           
    D_losses, G_losses = [], []
    print("epoch:{}".format(epoch))
    for batch_idx, (x, _) in enumerate(train_loader):     
        D_losses.append(D_train(x))
        G_losses.append(G_train(x))

    print('[%d/%d]: loss_d: %.3f, loss_g: %.3f' % (
            (epoch), n_epoch, torch.mean(torch.FloatTensor(D_losses)), torch.mean(torch.FloatTensor(G_losses))))

epoch:1
D torch.Size([100, 784]) torch.Size([100, 1])
D torch.Size([100, 784]) torch.Size([100, 1])
D torch.Size([100, 1])
G torch.Size([100, 100]) torch.Size([100, 1])
G_loss: tensor(1.3353, grad_fn=<BinaryCrossEntropyBackward>)
D torch.Size([100, 784]) torch.Size([100, 1])
D torch.Size([100, 784]) torch.Size([100, 1])
D torch.Size([100, 1])
G torch.Size([100, 100]) torch.Size([100, 1])
G_loss: tensor(1.6258, grad_fn=<BinaryCrossEntropyBackward>)
D torch.Size([100, 784]) torch.Size([100, 1])
D torch.Size([100, 784]) torch.Size([100, 1])
D torch.Size([100, 1])
G torch.Size([100, 100]) torch.Size([100, 1])
G_loss: tensor(1.6146, grad_fn=<BinaryCrossEntropyBackward>)
D torch.Size([100, 784]) torch.Size([100, 1])
D torch.Size([100, 784]) torch.Size([100, 1])
D torch.Size([100, 1])
G torch.Size([100, 100]) torch.Size([100, 1])
G_loss: tensor(1.8264, grad_fn=<BinaryCrossEntropyBackward>)
D torch.Size([100, 784]) torch.Size([100, 1])
D torch.Size([100, 784]) torch.Size([100, 1])
D torch.Size

D torch.Size([100, 784]) torch.Size([100, 1])
D torch.Size([100, 784]) torch.Size([100, 1])
D torch.Size([100, 1])
G torch.Size([100, 100]) torch.Size([100, 1])
G_loss: tensor(0.6096, grad_fn=<BinaryCrossEntropyBackward>)
D torch.Size([100, 784]) torch.Size([100, 1])
D torch.Size([100, 784]) torch.Size([100, 1])
D torch.Size([100, 1])
G torch.Size([100, 100]) torch.Size([100, 1])
G_loss: tensor(0.5275, grad_fn=<BinaryCrossEntropyBackward>)
D torch.Size([100, 784]) torch.Size([100, 1])
D torch.Size([100, 784]) torch.Size([100, 1])
D torch.Size([100, 1])
G torch.Size([100, 100]) torch.Size([100, 1])
G_loss: tensor(0.5386, grad_fn=<BinaryCrossEntropyBackward>)
D torch.Size([100, 784]) torch.Size([100, 1])
D torch.Size([100, 784]) torch.Size([100, 1])
D torch.Size([100, 1])
G torch.Size([100, 100]) torch.Size([100, 1])
G_loss: tensor(0.6256, grad_fn=<BinaryCrossEntropyBackward>)
D torch.Size([100, 784]) torch.Size([100, 1])
D torch.Size([100, 784]) torch.Size([100, 1])
D torch.Size([100, 1

D torch.Size([100, 784]) torch.Size([100, 1])
D torch.Size([100, 784]) torch.Size([100, 1])
D torch.Size([100, 1])
G torch.Size([100, 100]) torch.Size([100, 1])
G_loss: tensor(2.0714, grad_fn=<BinaryCrossEntropyBackward>)
D torch.Size([100, 784]) torch.Size([100, 1])
D torch.Size([100, 784]) torch.Size([100, 1])
D torch.Size([100, 1])
G torch.Size([100, 100]) torch.Size([100, 1])
G_loss: tensor(2.0425, grad_fn=<BinaryCrossEntropyBackward>)
D torch.Size([100, 784]) torch.Size([100, 1])
D torch.Size([100, 784]) torch.Size([100, 1])
D torch.Size([100, 1])
G torch.Size([100, 100]) torch.Size([100, 1])
G_loss: tensor(2.1079, grad_fn=<BinaryCrossEntropyBackward>)
D torch.Size([100, 784]) torch.Size([100, 1])
D torch.Size([100, 784]) torch.Size([100, 1])
D torch.Size([100, 1])
G torch.Size([100, 100]) torch.Size([100, 1])
G_loss: tensor(2.0126, grad_fn=<BinaryCrossEntropyBackward>)
D torch.Size([100, 784]) torch.Size([100, 1])
D torch.Size([100, 784]) torch.Size([100, 1])
D torch.Size([100, 1

D torch.Size([100, 1])
G torch.Size([100, 100]) torch.Size([100, 1])
G_loss: tensor(2.4293, grad_fn=<BinaryCrossEntropyBackward>)
D torch.Size([100, 784]) torch.Size([100, 1])
D torch.Size([100, 784]) torch.Size([100, 1])
D torch.Size([100, 1])
G torch.Size([100, 100]) torch.Size([100, 1])
G_loss: tensor(2.4603, grad_fn=<BinaryCrossEntropyBackward>)
D torch.Size([100, 784]) torch.Size([100, 1])
D torch.Size([100, 784]) torch.Size([100, 1])
D torch.Size([100, 1])
G torch.Size([100, 100]) torch.Size([100, 1])
G_loss: tensor(2.5585, grad_fn=<BinaryCrossEntropyBackward>)
D torch.Size([100, 784]) torch.Size([100, 1])
D torch.Size([100, 784]) torch.Size([100, 1])
D torch.Size([100, 1])
G torch.Size([100, 100]) torch.Size([100, 1])
G_loss: tensor(2.5619, grad_fn=<BinaryCrossEntropyBackward>)
D torch.Size([100, 784]) torch.Size([100, 1])
D torch.Size([100, 784]) torch.Size([100, 1])
D torch.Size([100, 1])
G torch.Size([100, 100]) torch.Size([100, 1])
G_loss: tensor(2.6232, grad_fn=<BinaryCross

G torch.Size([100, 100]) torch.Size([100, 1])
G_loss: tensor(2.0579, grad_fn=<BinaryCrossEntropyBackward>)
D torch.Size([100, 784]) torch.Size([100, 1])
D torch.Size([100, 784]) torch.Size([100, 1])
D torch.Size([100, 1])
G torch.Size([100, 100]) torch.Size([100, 1])
G_loss: tensor(1.8555, grad_fn=<BinaryCrossEntropyBackward>)
D torch.Size([100, 784]) torch.Size([100, 1])
D torch.Size([100, 784]) torch.Size([100, 1])
D torch.Size([100, 1])
G torch.Size([100, 100]) torch.Size([100, 1])
G_loss: tensor(1.7573, grad_fn=<BinaryCrossEntropyBackward>)
D torch.Size([100, 784]) torch.Size([100, 1])
D torch.Size([100, 784]) torch.Size([100, 1])
D torch.Size([100, 1])
G torch.Size([100, 100]) torch.Size([100, 1])
G_loss: tensor(1.5429, grad_fn=<BinaryCrossEntropyBackward>)
D torch.Size([100, 784]) torch.Size([100, 1])
D torch.Size([100, 784]) torch.Size([100, 1])
D torch.Size([100, 1])
G torch.Size([100, 100]) torch.Size([100, 1])
G_loss: tensor(1.5750, grad_fn=<BinaryCrossEntropyBackward>)
D tor

...

D torch.Size([100, 784]) torch.Size([100, 1])
D torch.Size([100, 1])
G torch.Size([100, 100]) torch.Size([100, 1])
G_loss: tensor(1.4938, grad_fn=<BinaryCrossEntropyBackward>)
D torch.Size([100, 784]) torch.Size([100, 1])
D torch.Size([100, 784]) torch.Size([100, 1])
D torch.Size([100, 1])
G torch.Size([100, 100]) torch.Size([100, 1])
G_loss: tensor(1.5549, grad_fn=<BinaryCrossEntropyBackward>)
D torch.Size([100, 784]) torch.Size([100, 1])
D torch.Size([100, 784]) torch.Size([100, 1])
D torch.Size([100, 1])
G torch.Size([100, 100]) torch.Size([100, 1])
G_loss: tensor(1.6082, grad_fn=<BinaryCrossEntropyBackward>)
D torch.Size([100, 784]) torch.Size([100, 1])
D torch.Size([100, 784]) torch.Size([100, 1])
D torch.Size([100, 1])
G torch.Size([100, 100]) torch.Size([100, 1])
G_loss: tensor(1.7949, grad_fn=<BinaryCrossEntropyBackward>)
D torch.Size([100, 784]) torch.Size([100, 1])
D torch.Size([100, 784]) torch.Size([100, 1])
D torch.Size([100, 1])
G torch.Size([100, 100]) torch.Size([100, 1

G torch.Size([100, 100]) torch.Size([100, 1])
G_loss: tensor(1.3090, grad_fn=<BinaryCrossEntropyBackward>)
D torch.Size([100, 784]) torch.Size([100, 1])
D torch.Size([100, 784]) torch.Size([100, 1])
D torch.Size([100, 1])
G torch.Size([100, 100]) torch.Size([100, 1])
G_loss: tensor(1.3142, grad_fn=<BinaryCrossEntropyBackward>)
D torch.Size([100, 784]) torch.Size([100, 1])
D torch.Size([100, 784]) torch.Size([100, 1])
D torch.Size([100, 1])
G torch.Size([100, 100]) torch.Size([100, 1])
G_loss: tensor(1.3285, grad_fn=<BinaryCrossEntropyBackward>)
D torch.Size([100, 784]) torch.Size([100, 1])
D torch.Size([100, 784]) torch.Size([100, 1])
D torch.Size([100, 1])
G torch.Size([100, 100]) torch.Size([100, 1])
G_loss: tensor(1.2188, grad_fn=<BinaryCrossEntropyBackward>)
D torch.Size([100, 784]) torch.Size([100, 1])
D torch.Size([100, 784]) torch.Size([100, 1])
D torch.Size([100, 1])
G torch.Size([100, 100]) torch.Size([100, 1])
G_loss: tensor(0.9567, grad_fn=<BinaryCrossEntropyBackward>)
D tor

D torch.Size([100, 784]) torch.Size([100, 1])
D torch.Size([100, 784]) torch.Size([100, 1])
D torch.Size([100, 1])
G torch.Size([100, 100]) torch.Size([100, 1])
G_loss: tensor(3.2242, grad_fn=<BinaryCrossEntropyBackward>)
D torch.Size([100, 784]) torch.Size([100, 1])
D torch.Size([100, 784]) torch.Size([100, 1])
D torch.Size([100, 1])
G torch.Size([100, 100]) torch.Size([100, 1])
G_loss: tensor(2.8069, grad_fn=<BinaryCrossEntropyBackward>)
D torch.Size([100, 784]) torch.Size([100, 1])
D torch.Size([100, 784]) torch.Size([100, 1])
D torch.Size([100, 1])
G torch.Size([100, 100]) torch.Size([100, 1])
G_loss: tensor(2.8367, grad_fn=<BinaryCrossEntropyBackward>)
D torch.Size([100, 784]) torch.Size([100, 1])
D torch.Size([100, 784]) torch.Size([100, 1])
D torch.Size([100, 1])
G torch.Size([100, 100]) torch.Size([100, 1])
G_loss: tensor(2.8781, grad_fn=<BinaryCrossEntropyBackward>)
D torch.Size([100, 784]) torch.Size([100, 1])
D torch.Size([100, 784]) torch.Size([100, 1])
D torch.Size([100, 1

D torch.Size([100, 784]) torch.Size([100, 1])
D torch.Size([100, 784]) torch.Size([100, 1])
D torch.Size([100, 1])
G torch.Size([100, 100]) torch.Size([100, 1])
G_loss: tensor(4.5621, grad_fn=<BinaryCrossEntropyBackward>)
D torch.Size([100, 784]) torch.Size([100, 1])
D torch.Size([100, 784]) torch.Size([100, 1])
D torch.Size([100, 1])
G torch.Size([100, 100]) torch.Size([100, 1])
G_loss: tensor(3.8353, grad_fn=<BinaryCrossEntropyBackward>)
D torch.Size([100, 784]) torch.Size([100, 1])
D torch.Size([100, 784]) torch.Size([100, 1])
D torch.Size([100, 1])
G torch.Size([100, 100]) torch.Size([100, 1])
G_loss: tensor(3.1867, grad_fn=<BinaryCrossEntropyBackward>)
D torch.Size([100, 784]) torch.Size([100, 1])
D torch.Size([100, 784]) torch.Size([100, 1])
D torch.Size([100, 1])
G torch.Size([100, 100]) torch.Size([100, 1])
G_loss: tensor(2.4884, grad_fn=<BinaryCrossEntropyBackward>)
D torch.Size([100, 784]) torch.Size([100, 1])
D torch.Size([100, 784]) torch.Size([100, 1])
D torch.Size([100, 1

G torch.Size([100, 100]) torch.Size([100, 1])
G_loss: tensor(1.8600, grad_fn=<BinaryCrossEntropyBackward>)
[2/2]: loss_d: 0.936, loss_g: 1.806


In [11]:
with torch.no_grad():
    test_z = Variable(torch.randn(bs, z_dim).to(device))
    generated = G(test_z)

    save_image(generated.view(generated.size(0), 1, 28, 28), './samples/sample_' + '.png')